In [ ]:
import pandas as pd
import talib
import json

# Substitua 'caminho_do_arquivo.csv' pelo caminho do seu arquivo CSV
caminho_do_arquivo = "C:/Projetos/USDJPYDaily.csv"

# Lê o arquivo CSV
df = pd.read_csv(caminho_do_arquivo)

# Converte a coluna de data para datetime
df['Date'] = pd.to_datetime(df['Date'])

# Filtra os anos de 2015 a 2025
df_filtered = df[(df['Date'].dt.year >= 2015) & (df['Date'].dt.year <= 2025)]

# Calcula o índice direcional médio de 14 períodos (ADX)
df_filtered['ADX'] = talib.ADX(df_filtered['High'], df_filtered['Low'], df_filtered['Close'], timeperiod=14)

# Calcula DI+ e DI-
df_filtered['DI+'] = talib.PLUS_DI(df_filtered['High'], df_filtered['Low'], df_filtered['Close'], timeperiod=14)
df_filtered['DI-'] = talib.MINUS_DI(df_filtered['High'], df_filtered['Low'], df_filtered['Close'], timeperiod=14)

# Determina o tipo de operação (compra ou venda)
df_filtered['Operation'] = df_filtered.apply(lambda row: 'Compra' if row['DI+'] > row['DI-'] else 'Venda', axis=1)

# Calcula o True Range (TR) de 10 períodos
df_filtered['TR'] = talib.ATR(df_filtered['High'], df_filtered['Low'], df_filtered['Close'], timeperiod=10)

# Calcula o stop e o gain
df_filtered['Stop'] = df_filtered['Close'] - 1.5 * df_filtered['TR']
df_filtered['Gain'] = df_filtered['Close'] + 4 * df_filtered['TR']

# Inicializa uma lista para armazenar os resultados das operações
operations = []

# Itera sobre as linhas do DataFrame para verificar condições de operação
for index, row in df_filtered.iterrows():
    if row['ADX'] > 30:  # Verifica se o ADX é maior que 30
        operation = {
            "Date": row['Date'],
            "Type": row['Operation'],
            "EntryPrice": row['Close'],
            "Stop": row['Stop'],
            "Gain": row['Gain'],
            "Result": None  # Inicializa o resultado como None
        }

        # Simula o preço atingindo o Gain ou o Stop
        # Substitua esta parte com dados reais de preços em tempo real, se disponível
        simulated_prices = [row['Close'], row['Gain'], row['Stop']]  # Exemplo de preços simulados
        for price in simulated_prices:
            if price >= row['Gain']:
                operation['Result'] = 'Gain'
                break
            elif price <= row['Stop']:
                operation['Result'] = 'Stop'
                break

        # Adiciona a operação à lista
        operations.append(operation)

# Salva os resultados das operações em um arquivo JSON
with open("operations_result.json", "w") as json_file:
    json.dump(operations, json_file, indent=4, default=str)

# Exibe o resultado das operações
print(operations)


In [24]:
import pandas as pd
import talib
import json

# Substitua 'caminho_do_arquivo.csv' pelo caminho do seu arquivo CSV
caminho_do_arquivo = "C:/Projetos/USDJPYDaily.csv"

# Configurações do mercado futuro
contract_size = 1000  # Tamanho do contrato (exemplo: 1000 unidades)
tick_value = 0.01     # Valor de cada ponto (exemplo: 0.01 por ponto)

# Lê o arquivo CSV
df = pd.read_csv(caminho_do_arquivo)

# Converte a coluna de data para datetime
df['Date'] = pd.to_datetime(df['Date'])

# Filtra os anos de 2015 a 2025
df_filtered = df[(df['Date'].dt.year >= 2015) & (df['Date'].dt.year <= 2025)]

# Calcula o índice direcional médio de 14 períodos (ADX)
df_filtered['ADX'] = talib.ADX(df_filtered['High'], df_filtered['Low'], df_filtered['Close'], timeperiod=14)

# Calcula DI+ e DI-
df_filtered['DI+'] = talib.PLUS_DI(df_filtered['High'], df_filtered['Low'], df_filtered['Close'], timeperiod=14)
df_filtered['DI-'] = talib.MINUS_DI(df_filtered['High'], df_filtered['Low'], df_filtered['Close'], timeperiod=14)

# Determina o tipo de operação (compra ou venda)
df_filtered['Operation'] = df_filtered.apply(lambda row: 'Compra' if row['DI+'] > row['DI-'] else 'Venda', axis=1)

# Calcula o True Range (TR) de 10 períodos
df_filtered['TR'] = talib.ATR(df_filtered['High'], df_filtered['Low'], df_filtered['Close'], timeperiod=10)

# Inicializa uma lista para armazenar os resultados das operações
operations = []

# Itera sobre as linhas do DataFrame para verificar condições de operação
for index, row in df_filtered.iterrows():
    if row['ADX'] > 30:  # Verifica se o ADX é maior que 30
        # Define os valores de Stop e Gain com base no tipo de operação
        if row['Operation'] == 'Compra':
            stop = row['Close'] - 1.5 * row['TR']
            gain = row['Close'] + 4 * row['TR']
        elif row['Operation'] == 'Venda':
            stop = row['Close'] + 1.5 * row['TR']
            gain = row['Close'] - 4 * row['TR']

        # Cria a operação
        operation = {
            "Date": row['Date'],
            "Type": row['Operation'],
            "EntryPrice": row['Close'],
            "Stop": stop,
            "Gain": gain,
            "Result": None,  # Inicializa o resultado como None
            "Profit/Loss": None  # Inicializa o lucro/prejuízo como None
        }

        # Simula o preço atingindo o Gain ou o Stop
        # Substitua esta parte com dados reais de preços em tempo real, se disponível
        simulated_prices = [row['Close'], gain, stop]  # Exemplo de preços simulados
        for price in simulated_prices:
            if row['Operation'] == 'Compra' and price >= gain:
                operation['Result'] = 'Gain'
                operation['Profit/Loss'] = (gain - row['Close']) * contract_size * tick_value
                break
            elif row['Operation'] == 'Compra' and price <= stop:
                operation['Result'] = 'Stop'
                operation['Profit/Loss'] = (stop - row['Close']) * contract_size * tick_value
                break
            elif row['Operation'] == 'Venda' and price <= gain:
                operation['Result'] = 'Gain'
                operation['Profit/Loss'] = (row['Close'] - gain) * contract_size * tick_value
                break
            elif row['Operation'] == 'Venda' and price >= stop:
                operation['Result'] = 'Stop'
                operation['Profit/Loss'] = (row['Close'] - stop) * contract_size * tick_value
                break

        # Adiciona a operação à lista
        operations.append(operation)

# Salva os resultados das operações em um arquivo JSON
with open("operations_result.json", "w") as json_file:
    json.dump(operations, json_file, indent=4, default=str)

# Exibe o resultado das operações
print(operations)

C:\Users\Arthur Age\AppData\Local\Temp\ipykernel_33692\1955062096.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['ADX'] = talib.ADX(df_filtered['High'], df_filtered['Low'], df_filtered['Close'], timeperiod=14)
C:\Users\Arthur Age\AppData\Local\Temp\ipykernel_33692\1955062096.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['DI+'] = talib.PLUS_DI(df_filtered['High'], df_filtered['Low'], df_filtered['Close'], timeperiod=14)
C:\Users\Arthur Age\AppData\Local\Temp\ipykernel_336

[{'Date': Timestamp('2015-02-10 00:00:00'), 'Type': 'Compra', 'EntryPrice': 119.42, 'Stop': 117.64219384814794, 'Gain': 124.16081640493886, 'Result': 'Gain', 'Profit/Loss': 47.408164049388546}, {'Date': Timestamp('2015-02-11 00:00:00'), 'Type': 'Compra', 'EntryPrice': 120.445, 'Stop': 118.66677446333313, 'Gain': 125.18693476444496, 'Result': 'Gain', 'Profit/Loss': 47.41934764444963}, {'Date': Timestamp('2015-02-12 00:00:00'), 'Type': 'Compra', 'EntryPrice': 119.102, 'Stop': 117.20819701699983, 'Gain': 124.15214128800048, 'Result': 'Gain', 'Profit/Loss': 50.50141288000476}, {'Date': Timestamp('2015-02-13 00:00:00'), 'Type': 'Compra', 'EntryPrice': 118.652, 'Stop': 116.82982731529984, 'Gain': 123.51112715920043, 'Result': 'Gain', 'Profit/Loss': 48.59127159200426}, {'Date': Timestamp('2015-02-16 00:00:00'), 'Type': 'Compra', 'EntryPrice': 118.47, 'Stop': 116.75579458376986, 'Gain': 123.04121444328038, 'Result': 'Gain', 'Profit/Loss': 45.712144432803825}, {'Date': Timestamp('2015-02-17 00:

In [25]:
import pandas as pd
import talib
import json

# Substitua 'caminho_do_arquivo.csv' pelo caminho do seu arquivo CSV
caminho_do_arquivo = "C:/Projetos/USDJPYDaily.csv"

# Configurações do mercado futuro
contract_size = 1000  # Tamanho do contrato (exemplo: 1000 unidades)
tick_value = 0.01     # Valor de cada ponto (exemplo: 0.01 por ponto)

# Lê o arquivo CSV
df = pd.read_csv(caminho_do_arquivo)

# Converte a coluna de data para datetime
df['Date'] = pd.to_datetime(df['Date'])

# Filtra os anos de 2015 a 2025
df_filtered = df[(df['Date'].dt.year >= 2015) & (df['Date'].dt.year <= 2025)]

# Calcula o índice direcional médio de 14 períodos (ADX)
df_filtered['ADX'] = talib.ADX(df_filtered['High'], df_filtered['Low'], df_filtered['Close'], timeperiod=14)

# Calcula DI+ e DI-
df_filtered['DI+'] = talib.PLUS_DI(df_filtered['High'], df_filtered['Low'], df_filtered['Close'], timeperiod=14)
df_filtered['DI-'] = talib.MINUS_DI(df_filtered['High'], df_filtered['Low'], df_filtered['Close'], timeperiod=14)

# Determina o tipo de operação (compra ou venda)
df_filtered['Operation'] = df_filtered.apply(lambda row: 'Compra' if row['DI+'] > row['DI-'] else 'Venda', axis=1)

# Calcula o True Range (TR) de 10 períodos
df_filtered['TR'] = talib.ATR(df_filtered['High'], df_filtered['Low'], df_filtered['Close'], timeperiod=10)

# Inicializa uma lista para armazenar os resultados das operações
operations = []

# Itera sobre as linhas do DataFrame para verificar condições de operação
for index, row in df_filtered.iterrows():
    if row['ADX'] > 30:  # Verifica se o ADX é maior que 30
        # Define os valores de Stop e Gain com base no tipo de operação
        if row['Operation'] == 'Compra':
            stop = row['Close'] - 1.5 * row['TR']
            gain = row['Close'] + 4 * row['TR']
        elif row['Operation'] == 'Venda':
            stop = row['Close'] + 1.5 * row['TR']
            gain = row['Close'] - 4 * row['TR']

        # Cria a operação
        operation = {
            "Date": row['Date'],
            "Type": row['Operation'],
            "EntryPrice": row['Close'],
            "Stop": stop,
            "Gain": gain,
            "Result": None,  # Inicializa o resultado como None
            "Profit/Loss": None  # Inicializa o lucro/prejuízo como None
        }

        # Simula o preço atingindo o Gain ou o Stop
        # Substitua esta parte com dados reais de preços em tempo real, se disponível
        simulated_prices = [row['Close'], stop, gain]  # Verifica o Stop antes do Gain
        for price in simulated_prices:
            if row['Operation'] == 'Compra' and price <= stop:
                operation['Result'] = 'Stop'
                operation['Profit/Loss'] = (stop - row['Close']) * contract_size * tick_value
                break
            elif row['Operation'] == 'Compra' and price >= gain:
                operation['Result'] = 'Gain'
                operation['Profit/Loss'] = (gain - row['Close']) * contract_size * tick_value
                break
            elif row['Operation'] == 'Venda' and price >= stop:
                operation['Result'] = 'Stop'
                operation['Profit/Loss'] = (row['Close'] - stop) * contract_size * tick_value
                break
            elif row['Operation'] == 'Venda' and price <= gain:
                operation['Result'] = 'Gain'
                operation['Profit/Loss'] = (row['Close'] - gain) * contract_size * tick_value
                break

        # Adiciona a operação à lista
        operations.append(operation)

# Salva os resultados das operações em um arquivo JSON
with open("operations_result.json", "w") as json_file:
    json.dump(operations, json_file, indent=4, default=str)

# Exibe o resultado das operações
print(operations)

C:\Users\Arthur Age\AppData\Local\Temp\ipykernel_33692\3579065754.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['ADX'] = talib.ADX(df_filtered['High'], df_filtered['Low'], df_filtered['Close'], timeperiod=14)
C:\Users\Arthur Age\AppData\Local\Temp\ipykernel_33692\3579065754.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['DI+'] = talib.PLUS_DI(df_filtered['High'], df_filtered['Low'], df_filtered['Close'], timeperiod=14)
C:\Users\Arthur Age\AppData\Local\Temp\ipykernel_336

[{'Date': Timestamp('2015-02-10 00:00:00'), 'Type': 'Compra', 'EntryPrice': 119.42, 'Stop': 117.64219384814794, 'Gain': 124.16081640493886, 'Result': 'Stop', 'Profit/Loss': -17.778061518520616}, {'Date': Timestamp('2015-02-11 00:00:00'), 'Type': 'Compra', 'EntryPrice': 120.445, 'Stop': 118.66677446333313, 'Gain': 125.18693476444496, 'Result': 'Stop', 'Profit/Loss': -17.782255366668664}, {'Date': Timestamp('2015-02-12 00:00:00'), 'Type': 'Compra', 'EntryPrice': 119.102, 'Stop': 117.20819701699983, 'Gain': 124.15214128800048, 'Result': 'Stop', 'Profit/Loss': -18.93802983000171}, {'Date': Timestamp('2015-02-13 00:00:00'), 'Type': 'Compra', 'EntryPrice': 118.652, 'Stop': 116.82982731529984, 'Gain': 123.51112715920043, 'Result': 'Stop', 'Profit/Loss': -18.221726847001634}, {'Date': Timestamp('2015-02-16 00:00:00'), 'Type': 'Compra', 'EntryPrice': 118.47, 'Stop': 116.75579458376986, 'Gain': 123.04121444328038, 'Result': 'Stop', 'Profit/Loss': -17.142054162301434}, {'Date': Timestamp('2015-02

In [26]:
import pandas as pd
import talib
import json

# Substitua 'caminho_do_arquivo.csv' pelo caminho do seu arquivo CSV
caminho_do_arquivo = "C:/Projetos/USDJPYDaily.csv"

# Configurações do mercado futuro
contract_size = 1000  # Tamanho do contrato (exemplo: 1000 unidades)
tick_value = 0.01     # Valor de cada ponto (exemplo: 0.01 por ponto)

# Lê o arquivo CSV
df = pd.read_csv(caminho_do_arquivo)

# Converte a coluna de data para datetime
df['Date'] = pd.to_datetime(df['Date'])

# Filtra os anos de 2015 a 2025
df_filtered = df[(df['Date'].dt.year >= 2015) & (df['Date'].dt.year <= 2025)]

# Calcula o índice direcional médio de 14 períodos (ADX)
df_filtered['ADX'] = talib.ADX(df_filtered['High'], df_filtered['Low'], df_filtered['Close'], timeperiod=14)

# Calcula DI+ e DI-
df_filtered['DI+'] = talib.PLUS_DI(df_filtered['High'], df_filtered['Low'], df_filtered['Close'], timeperiod=14)
df_filtered['DI-'] = talib.MINUS_DI(df_filtered['High'], df_filtered['Low'], df_filtered['Close'], timeperiod=14)

# Determina o tipo de operação (compra ou venda)
df_filtered['Operation'] = df_filtered.apply(lambda row: 'Compra' if row['DI+'] > row['DI-'] else 'Venda', axis=1)

# Calcula o True Range (TR) de 10 períodos
df_filtered['TR'] = talib.ATR(df_filtered['High'], df_filtered['Low'], df_filtered['Close'], timeperiod=10)

# Inicializa uma lista para armazenar os resultados das operações
operations = []

# Itera sobre as linhas do DataFrame para verificar condições de operação
for index, row in df_filtered.iterrows():
    if row['ADX'] > 30:  # Verifica se o ADX é maior que 30
        # Define os valores de Stop e Gain com base no tipo de operação
        if row['Operation'] == 'Compra':
            stop = row['Close'] - 1.5 * row['TR']
            gain = row['Close'] + 4 * row['TR']
        elif row['Operation'] == 'Venda':
            stop = row['Close'] + 1.5 * row['TR']
            gain = row['Close'] - 4 * row['TR']

        # Cria a operação
        operation = {
            "Date": row['Date'],
            "Type": row['Operation'],
            "EntryPrice": row['Close'],
            "Stop": stop,
            "Gain": gain,
            "Result": None,  # Inicializa o resultado como None
            "Profit/Loss": None  # Inicializa o lucro/prejuízo como None
        }

        # Simula o preço atingindo o Gain ou o Stop
        # Substitua esta parte com dados reais de preços em tempo real, se disponível
        simulated_prices = [row['Close'], gain, stop]  # Verifica o Gain antes do Stop
        for price in simulated_prices:
            if row['Operation'] == 'Compra' and price >= gain:
                operation['Result'] = 'Gain'
                operation['Profit/Loss'] = (gain - row['Close']) * contract_size * tick_value
                break
            elif row['Operation'] == 'Compra' and price <= stop:
                operation['Result'] = 'Stop'
                operation['Profit/Loss'] = (stop - row['Close']) * contract_size * tick_value
                break
            elif row['Operation'] == 'Venda' and price <= gain:
                operation['Result'] = 'Gain'
                operation['Profit/Loss'] = (row['Close'] - gain) * contract_size * tick_value
                break
            elif row['Operation'] == 'Venda' and price >= stop:
                operation['Result'] = 'Stop'
                operation['Profit/Loss'] = (row['Close'] - stop) * contract_size * tick_value
                break

        # Adiciona a operação à lista
        operations.append(operation)

# Salva os resultados das operações em um arquivo JSON
with open("operations_result.json", "w") as json_file:
    json.dump(operations, json_file, indent=4, default=str)

# Exibe o resultado das operações
print(operations)

C:\Users\Arthur Age\AppData\Local\Temp\ipykernel_33692\2032247024.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['ADX'] = talib.ADX(df_filtered['High'], df_filtered['Low'], df_filtered['Close'], timeperiod=14)
C:\Users\Arthur Age\AppData\Local\Temp\ipykernel_33692\2032247024.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['DI+'] = talib.PLUS_DI(df_filtered['High'], df_filtered['Low'], df_filtered['Close'], timeperiod=14)
C:\Users\Arthur Age\AppData\Local\Temp\ipykernel_336

[{'Date': Timestamp('2015-02-10 00:00:00'), 'Type': 'Compra', 'EntryPrice': 119.42, 'Stop': 117.64219384814794, 'Gain': 124.16081640493886, 'Result': 'Gain', 'Profit/Loss': 47.408164049388546}, {'Date': Timestamp('2015-02-11 00:00:00'), 'Type': 'Compra', 'EntryPrice': 120.445, 'Stop': 118.66677446333313, 'Gain': 125.18693476444496, 'Result': 'Gain', 'Profit/Loss': 47.41934764444963}, {'Date': Timestamp('2015-02-12 00:00:00'), 'Type': 'Compra', 'EntryPrice': 119.102, 'Stop': 117.20819701699983, 'Gain': 124.15214128800048, 'Result': 'Gain', 'Profit/Loss': 50.50141288000476}, {'Date': Timestamp('2015-02-13 00:00:00'), 'Type': 'Compra', 'EntryPrice': 118.652, 'Stop': 116.82982731529984, 'Gain': 123.51112715920043, 'Result': 'Gain', 'Profit/Loss': 48.59127159200426}, {'Date': Timestamp('2015-02-16 00:00:00'), 'Type': 'Compra', 'EntryPrice': 118.47, 'Stop': 116.75579458376986, 'Gain': 123.04121444328038, 'Result': 'Gain', 'Profit/Loss': 45.712144432803825}, {'Date': Timestamp('2015-02-17 00: